In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-09"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11308,2024-05-09,Paraguai Lnb,20:30,Ciudad Nueva,Dep. San Jose,14.50,28.00,149.5,1.85,1.83,0.0,0.00,0.0,nu7OKtK8,0.068966,0.035714,0.540541,0.546448,-0.895320,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,58,73,1.09,1.05,0.000,0.000,0.449221,0.007686,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
11309,2024-05-09,Polônia Liga De Basquete,15:45,Slask Wroclaw,Ostrow Wielkopolski,1.71,2.07,162.5,1.87,1.87,-2.5,1.94,2.5,G0jtyiEN,0.584795,0.483092,0.534759,0.534759,0.067887,160.416,72.858347,0.454184,1.8,1.643168,0.912871,205.44,2.106,1.009222,0.479213,-9.0,130.070,27.118684,0.208493,2.4,1.341641,0.559017,107.73,1.410,0.290689,0.206162,79.0,64,81,3.21,1.33,142.938,171.034,0.134687,0.000000,0.178369,-0.64,-0.128,-5.546875,0.634564,0.6,-0.034564,0.56,0.112,9.553571,0.608768,0.6,-0.008768
11310,2024-05-09,Polônia Liga De Basquete,15:00,Legia,Szczecin,1.41,2.82,168.5,1.87,1.86,0.0,0.00,0.0,n7JjQZa5,0.709220,0.354610,0.534759,0.537634,0.063830,179.226,12.592024,0.070258,1.8,1.643168,0.912871,165.55,2.026,0.306643,0.151354,22.0,165.818,44.269659,0.266977,2.4,1.341641,0.559017,133.60,1.854,0.332461,0.179321,22.0,77,80,2.15,1.67,135.688,194.230,0.471405,0.003791,NaN,-0.51,-0.102,-4.019608,0.706715,0.5,-0.206715,5.53,1.106,1.645570,0.571541,1.0,0.428459
11311,2024-05-09,Polônia Liga De Basquete,13:00,Spojnia Stargard,Anwil Wloclawek,2.60,1.47,158.5,1.88,1.86,0.0,0.00,0.0,fDtItepn,0.384615,0.680272,0.531915,0.537634,0.064887,190.636,77.742246,0.407805,0.6,1.341641,2.236068,269.64,2.296,0.992890,0.432443,-16.0,136.250,28.716953,0.210767,1.8,1.643168,0.912871,130.34,1.516,0.364459,0.240408,43.0,84,98,3.21,1.33,123.488,150.030,0.392644,0.007563,NaN,-1.09,-0.218,-7.339450,0.710607,0.8,0.089393,-0.16,-0.032,-14.687500,0.641952,0.6,-0.041952
11312,2024-05-09,Paraguai Lnb,20:30,Libertad,Sol De America,1.07,7.28,143.5,1.85,1.85,0.0,0.00,0.0,hhxPIKlL,0.934579,0.137363,0.540541,0.540541,0.071942,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,73,51,4.42,6.52,0.000,0.000,1.051768,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11367,2024-05-09,Geórgia Superliga,10:00,Gurjaani Delta,Kavkasia,1.33,3.12,165.5,1.85,1.85,0.0,0.00,0.0,A1xHUWHq,0.751880,0.320513,0.540541,0.540541,0.072393,118.408,15.538699,0.131230,1.8,1.643168,0.912871,145.25,1.496,0.156621,0.104693,0.0,213.246,31.565970,0.148026,1.8,1.643168,0.912871,253.44,2.336,0.243578,0.104271,-4.0,83,99,1.75,2.56,118.548,210.198,0.568863,0.000000,NaN,-0.91,-0.182,-1.813187,0.000000,0.0,0.000000,2.41,0.482,4.398340,0.000000,0.0,0.000000
11368,2024-05-09,Geórgia Superliga,12:00,Kutaisi,Gau,1.35,3.02,176.5,1.85,1.85,0.0,0.00,0.0,pxpVwyec,0.740741,0.331126,0.540541,0.540541,0.071867,129.516,23.525486,0.181642,2.4,1.341641,0.559017,109.20,1.322,0.118406,0.089566,52.0

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
11364,15:15,Croácia Premijer Liga,Cedevita,Dinamo Zagreb,1.68,Back Home
